In [1]:
# %load_ext autoreload
# %autoreload 2
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score
import sys
sys.path.append(os.path.abspath(".."))
import torch
import pandas as pd
from datasets import PressingSequenceDataset, SoccerMapInputDataset
import config as C
import features as F
from bisect import bisect_right
from collections import defaultdict


2025-06-14 07:52:00.270246: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-14 07:52:00.270294: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-14 07:52:00.270327: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 07:52:00.278843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-14 07:52:01.241611: W tensorflow/compiler/

/home/exPress/express-v2


In [2]:
data_path = "/home/exPress/express-v2/data/bepro/pressing_intensity"

with open(f"{data_path}/train_dataset.pkl", "rb") as f:
    train_dataset = pickle.load(f)

# with open(f"{data_path}/valid_dataset.pkl", "rb") as f:
#     valid_dataset = pickle.load(f)


with open(f"{data_path}/test_dataset.pkl", "rb") as f:
    test_dataset = pickle.load(f)

len(train_dataset), len(test_dataset)

(6569, 637)

In [15]:
def check_game_ids(dataset):
    game_ids = set()
    for idx, sample in enumerate(dataset):
        game_id, _, _ = sample['match_info'].split('-')
        game_ids.add(game_id)
    return len(game_ids), sorted(game_ids)

print("Train Game IDs:", check_game_ids(train_dataset))
print("Test Game IDs:", check_game_ids(test_dataset))


Train Game IDs: (34, ['126285', '126293', '126298', '126306', '126309', '126315', '126319', '126325', '126332', '126341', '126348', '126350', '126356', '126364', '126367', '126378', '126380', '126386', '126391', '126401', '126408', '126411', '126418', '126424', '126429', '126433', '126444', '126448', '126455', '126458', '126466', '126473', '126476', '153364'])
Test Game IDs: (3, ['153381', '153385', '153387'])


In [5]:
sample = train_dataset[0]

print(f"Features : {sample['features'].shape}")
print(f"Pressing Intensity : {sample['pressing_intensity'].shape}")
print(f"Labels : {sample['label']}")
print(f"Presser ID : {sample['presser_id']}")
print(f"Players Order : {sample['agent_order']}")
print(f"match info : {sample['match_info']}")

Features : torch.Size([3, 23, 18])
Pressing Intensity : torch.Size([3, 11, 11])
Labels : 0
Presser ID : 248890
Players Order : ['250079', '250101', '250102', '500133', '500135', '500139', '500140', '500141', '62365', '62386', '77414', '139210', '143410', '161110', '187326', '188266', '248890', '344466', '344467', '62009', '62038', '77579', 'ball']
match info : 126285-1.0-522


In [3]:
from collections import defaultdict
import torch
import pandas as pd
import os

def compute_second_half_offset(tracking_df):

    df = tracking_df.copy()
    df['timestamp'] = pd.to_timedelta(df['timestamp'])

    # 전반 종료 시점
    first_half_end = df[df['period_id'] == 1]['timestamp'].max()

    # 후반 시작 시점
    second_half_start = df[df['period_id'] == 2]['timestamp'].min()

    # offset 계산 (후반 timestamp에 더할 시간)
    offset = (first_half_end - second_half_start).total_seconds()

    return offset

def update_features(dataset, feature_list, processed_data_path, cache_dir="feature_cache"):
    os.makedirs(cache_dir, exist_ok=True)

    game_sample_map = defaultdict(list)

    # 1. game_id 기준으로 샘플 인덱스와 frame_id를 모음
    for idx, sample in enumerate(dataset): # dataset.__len__에 의해 결정되는 길이만큼 루프
        game_id, _, frame_id_str = sample['match_info'].split('-')
        frame_id = int(frame_id_str)
        game_sample_map[game_id].append((idx, frame_id))

    # 2. game_id 단위로 모든 처리 (데이터 로드, timestamp 매핑, 피처 계산, 데이터셋 업데이트)
    for game_id, frame_infos_for_game in game_sample_map.items():
        print(f"\n--- Processing Game ID: {game_id} ---")

        # 2-1. 해당 game_id의 데이터 로드
        tracking_df = pd.read_csv(os.path.join(processed_data_path, game_id, f"{game_id}_traces.csv"))
        events_df = pd.read_csv(os.path.join(processed_data_path, game_id, f"{game_id}_merged.csv"))
        teams_df = pd.read_csv(os.path.join(processed_data_path, game_id, f"{game_id}_teams.csv"))
        events_df = events_df.sort_values("timestamp")
        events_df["time_seconds"] = pd.to_numeric(events_df["time_seconds"], errors="coerce")

        offset = compute_second_half_offset(tracking_df)

        # 2-2. frame_id -> timestamp 매핑 (스코프가 이 루프 내부로 제한됨)
        frame_id_to_ts = {}
        for fid in [fid for _, fid in frame_infos_for_game]: # 현재 game_id에 해당하는 fid들만 사용
            row = tracking_df[tracking_df["frame_id"] == fid]
            if not row.empty:
                ts = pd.to_timedelta(row.iloc[0]["timestamp"]).total_seconds()
                if row.iloc[0]["period_id"] == 2:
                    ts += offset
                frame_id_to_ts[fid] = ts

        # 2-3. 피처 계산 및 버퍼링 (현재 game_id에 속하는 샘플들만 처리)
        # 이 버퍼는 현재 game_id 내에서만 사용됩니다.
        current_game_sample_feature_buffer = defaultdict(list) 

        for feature_name in feature_list: # 각 피처 이름에 대해
            func = getattr(F, feature_name)
            game_feature_dir = os.path.join(cache_dir, game_id)
            os.makedirs(game_feature_dir, exist_ok=True)

            cache_path = os.path.join(cache_dir, game_id, f"{game_id}_{feature_name}.npy")
            feature_cache = None
            
            if os.path.exists(cache_path):                
                feature_cache = np.load(cache_path, allow_pickle=True).item()  # dict: sample_idx -> (B, 1)
            else:
                feature_cache = {}

            feature_updated = False

            # 현재 game_id의 샘플들만 순회
            for sample_idx, fid in frame_infos_for_game: 
                if fid not in frame_id_to_ts:
                    print(f"WARN: Skipping sample_idx {sample_idx}, fid {fid} for game {game_id} as timestamp not found.")
                    continue
                ts = frame_id_to_ts[fid]
                current_sample_data_dict = dataset[sample_idx] 
                feature_tensor = current_sample_data_dict['features'] 
                B, N, D = feature_tensor.shape

                if sample_idx in feature_cache:
                    feat_val = feature_cache[sample_idx]
                else:                
                    full_past_events = events_df[events_df["time_seconds"] < ts].sort_values("time_seconds", ascending=False)
                    past_events = full_past_events.head(B)

                    if past_events.empty:
                        feat_val = np.full((B, 1), np.nan, dtype=np.float32)
                    else:
                        if feature_name == "sum_pitch_control":
                            feat_val = func(past_events, teams_df)                    
                        elif feature_name in ["time_since_last_opponent_action", "cumul_goal_att", "cumul_goal_def", "goaldiff"]:
                            feat_val = func(past_events, events_df)
                        else:
                            feat_val = func(past_events)
                    
                        if isinstance(feat_val, pd.DataFrame):
                            feat_val = feat_val.values.astype(np.float32)
                        elif isinstance(feat_val, np.ndarray):
                            feat_val = feat_val.astype(np.float32)
                    
                        if feat_val.ndim == 1:
                            feat_val = feat_val.reshape(-1, 1) # 항상 (B, 1) 또는 (이벤트 수, 1) 형태로 유지

                    feature_cache[sample_idx] = feat_val
                    feature_updated = True

                new_feature_tensor = torch.tensor(feat_val, dtype=torch.float32).unsqueeze(1).expand(-1, N, -1) # (B, N, F)
                current_game_sample_feature_buffer[sample_idx].append(new_feature_tensor)
                # print(f"DEBUG: New feature '{feature_name}' for sample {sample_idx} shape: {new_feature_tensor.shape}", flush=True)
            
            if feature_updated:
                np.save(cache_path, feature_cache)
                print(f"Saved cached feature for {feature_name} (game {game_id})")

        # 2-4. 현재 game_id에 대한 모든 피처 계산이 끝나면, dataset에 최종적으로 업데이트
        for sample_idx, new_feats_for_sample in current_game_sample_feature_buffer.items():
            # dataset에서 해당 sample_idx의 현재 데이터 딕셔너리를 가져옵니다.
            current_sample_data_dict = dataset[sample_idx] # __getitem__ 호출

            original_feature_tensor = current_sample_data_dict['features'] # 현재 features 텐서

            # 새롭게 추가될 모든 피처 텐서들을 dim=-1을 기준으로 횡으로 합칩니다.
            concat_all_new_features = torch.cat(new_feats_for_sample, dim=-1) 

            # 기존 피처 텐서와 새로 합쳐진 피처 텐서를 다시 dim=-1을 기준으로 합칩니다.
            final_concatenated_feature_tensor = torch.cat([original_feature_tensor, concat_all_new_features], dim=-1)

            # 업데이트된 features 텐서를 딕셔너리에 다시 할당합니다.
            current_sample_data_dict['features'] = final_concatenated_feature_tensor

            # dataset의 __setitem__을 호출하여 데이터셋 내부의 실제 데이터를 업데이트합니다.
            # 이 라인이 실행될 때 __setitem__이 없으면 TypeError가 발생하고,
            # __setitem__이 잘못 구현되면 IndexError가 발생합니다.
            dataset[sample_idx] = current_sample_data_dict 

            # print(f"DEBUG: Final shape for sample {sample_idx} in game {game_id}: {dataset[sample_idx]['features'].shape}", flush=True)
            # 이 print는 __getitem__이 호출되어 반환된 값을 보여줍니다.




In [17]:
def print_feature_shape_diff(dataset, dataset_name="dataset"):
    print(f"[{dataset_name}] Feature shape before update:")
    for i in range(3):  # 앞의 3개 샘플만 예시로 확인
        print(f"  Sample {i}: {dataset[i]['features'].shape}")

# 1. 업데이트 전
print_feature_shape_diff(train_dataset, "train_dataset (before)")
print_feature_shape_diff(test_dataset, "test_dataset (before)")

[train_dataset (before)] Feature shape before update:
  Sample 0: torch.Size([3, 23, 18])
  Sample 1: torch.Size([4, 23, 18])
  Sample 2: torch.Size([2, 23, 18])
[test_dataset (before)] Feature shape before update:
  Sample 0: torch.Size([2, 23, 18])
  Sample 1: torch.Size([2, 23, 18])
  Sample 2: torch.Size([1, 23, 18])


In [4]:
processed_data_path = "/home/exPress/express-v2/data/bepro/processed"
feature_list = [
                # 'distance_ball_goal','distance_ball_sideline', 'distance_ball_goalline', 'get_actor_speed', 'angle_to_goal', 
                # 'compute_time_elapsed','get_min_defender_distance', 'get_closest_defender_speed', 'get_speed_diff_actor_defender',
                # 'nb_of_3m_radius', 'nb_of_5m_radius','nb_of_10m_radius', 'get_dist_defender_to_sideline','get_dist_defender_to_goaline',
                'sum_pitch_control'
                ]

update_features(train_dataset, feature_list, processed_data_path, cache_dir="/home/exPress/express-v2/data/bepro/feature")
update_features(test_dataset, feature_list, processed_data_path, cache_dir="/home/exPress/express-v2/data/bepro/feature")


--- Processing Game ID: 126293 ---


100%|██████████| 2/2 [00:00<00:00, 10305.42it/s]
/home/exPress/express-v2/datatools/pitch_control.py:245: RuntimeWarning: divide by zero encountered in scalar divide
  checksum = np.sum( PPCFa + PPCFd ) / float(count)#float(n_grid_cells_y*n_grid_cells_x )
/home/exPress/express-v2/datatools/pitch_control.py:245: RuntimeWarning: divide by zero encountered in scalar divide
  checksum = np.sum( PPCFa + PPCFd ) / float(count)#float(n_grid_cells_y*n_grid_cells_x )
100%|██████████| 1/1 [00:00<00:00, 4534.38it/s]
/home/exPress/express-v2/datatools/pitch_control.py:245: RuntimeWarning: divide by zero encountered in scalar divide
  checksum = np.sum( PPCFa + PPCFd ) / float(count)#float(n_grid_cells_y*n_grid_cells_x )
100%|██████████| 1/1 [00:00<00:00, 6864.65it/s]
/home/exPress/express-v2/datatools/pitch_control.py:245: RuntimeWarning: divide by zero encountered in scalar divide
  checksum = np.sum( PPCFa + PPCFd ) / float(count)#float(n_grid_cells_y*n_grid_cells_x )
100%|██████████| 3/3 [00:00

In [11]:
data =pd.read_csv('/home/exPress/express-v2/data/bepro/processed/126298/126298_merged.csv')

In [12]:
data

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,relative_player_id,reactor_team_id,reactor_player_id,type_name,...,H06_y,H07_y,H08_y,H09_y,H10_y,H14_y,H16_y,H18_y,H19_y,ball_y
0,126298,0.0,0,1,0.967,2.067,407605.0,-1,-1,Pass,...,12.408782,-1.133164,-30.381690,0.282555,-10.505899,NaN,NaN,NaN,NaN,0.203899
1,126298,1.0,1,1,2.067,NaN,-1.0,-1,-1,Pass Received,...,12.013645,-1.181525,-30.856145,0.239845,-9.762821,NaN,NaN,NaN,NaN,-0.721556
2,126298,2.0,2,1,3.033,4.200,345446.0,-1,-1,Pass,...,13.059672,-1.067999,-31.521623,1.178115,-9.355349,NaN,NaN,NaN,NaN,0.045211
3,126298,3.0,3,1,4.200,NaN,-1.0,-1,-1,Pass Received,...,14.822376,-2.185743,-31.445433,1.805146,-8.132191,NaN,NaN,NaN,NaN,15.625385
4,126298,4.0,4,1,5.485,7.533,250899.0,-1,-1,Pass,...,14.219126,-4.111640,-30.602459,3.593756,-7.740082,NaN,NaN,NaN,NaN,13.541895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,126298,3525.0,2962,2,6501.767,NaN,NaN,4648,62365,Duel,...,-1.870507,NaN,NaN,NaN,-10.206191,-5.001248,-10.750800,-11.346866,0.788394,-13.771624
2963,126298,3529.0,2963,2,6503.855,NaN,-1.0,-1,-1,Recovery,...,-2.604813,NaN,NaN,NaN,-18.163922,-8.160125,-20.270517,-16.488018,-2.572863,-22.642875
2964,126298,NaN,2964,2,6503.856,NaN,NaN,-1,-1,Carry,...,-2.604813,NaN,NaN,NaN,-18.163922,-8.160125,-20.270517,-16.488018,-2.572863,-22.642875
2965,126298,3530.0,2965,2,6505.567,NaN,-1.0,-1,-1,Pass,...,-2.269982,NaN,NaN,NaN,-24.757684,-11.648197,-26.699439,-22.051588,-4.830030,-28.950543


In [6]:
num_seq = 150
num_agents = 11
use_pressing_intensity = True
selected_features_idx = [i for i in range(18)]

train_features = []
train_labels = []
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-1:]
    press_intensity = sample['pressing_intensity'][-1:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
            
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    train_features.append(feature_vector)
    train_labels.append(y_tensor.item())


test_features = []
test_labels = []
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-1:]
    press_intensity = sample['pressing_intensity'][-1:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    test_features.append(feature_vector)
    test_labels.append(y_tensor.item())

X_train = np.array(train_features)
y_train = np.array(train_labels)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_test = np.array(test_features)
y_test = np.array(test_labels)

In [7]:
x_tensor.shape 

torch.Size([1, 23, 18])

In [8]:
def print_dataset_distribution(y_train, y_val, y_test):
    def _print_split(name, labels):
        total = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        print(f"{name} Set:")
        print(f"  Total samples: {total}")
        for label, count in zip(unique, counts):
            percent = (count / total) * 100
            print(f"    Label {label}: {count:>5} samples ({percent:5.2f}%)")
        print("-" * 40)

    print("\n📊 Dataset Distribution Summary")
    print("=" * 40)
    _print_split("Train", y_train)
    _print_split("Validation", y_val)
    _print_split("Test", y_test)

print_dataset_distribution(y_train, y_val, y_test)


📊 Dataset Distribution Summary
Train Set:
  Total samples: 5922
    Label 0:  4361 samples (73.64%)
    Label 1:  1561 samples (26.36%)
----------------------------------------
Validation Set:
  Total samples: 1481
    Label 0:  1101 samples (74.34%)
    Label 1:   380 samples (25.66%)
----------------------------------------
Test Set:
  Total samples: 658
    Label 0:   547 samples (83.13%)
    Label 1:   111 samples (16.87%)
----------------------------------------


In [9]:
# Create XGBoost DMatrix objects for train and test sets
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

# Set XGBoost training parameters
params = {
    'objective': 'binary:logistic',  # binary classification
    'eval_metric': 'auc',            # evaluation metric: AUC
    'max_depth': 6,                  # maximum depth of trees
    'eta': 0.1,                      # learning rate
    'seed': 42
}

# Specify the watchlist to evaluate performance on training and test sets during training
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_rounds = 100

In [10]:
bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

[0]	train-auc:0.76474	eval-auc:0.58872


/opt/conda/envs/expressv2/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[1]	train-auc:0.79538	eval-auc:0.60409
[2]	train-auc:0.81498	eval-auc:0.63239
[3]	train-auc:0.83127	eval-auc:0.64188
[4]	train-auc:0.83337	eval-auc:0.64854
[5]	train-auc:0.83850	eval-auc:0.65784
[6]	train-auc:0.84749	eval-auc:0.65705
[7]	train-auc:0.86106	eval-auc:0.66110
[8]	train-auc:0.86346	eval-auc:0.66360
[9]	train-auc:0.87747	eval-auc:0.66454
[10]	train-auc:0.89042	eval-auc:0.66611
[11]	train-auc:0.89713	eval-auc:0.66332
[12]	train-auc:0.90123	eval-auc:0.66548
[13]	train-auc:0.90507	eval-auc:0.66677
[14]	train-auc:0.91075	eval-auc:0.66829
[15]	train-auc:0.91485	eval-auc:0.66997
[16]	train-auc:0.91735	eval-auc:0.67385
[17]	train-auc:0.92745	eval-auc:0.67296
[18]	train-auc:0.92733	eval-auc:0.67467
[19]	train-auc:0.93228	eval-auc:0.67536
[20]	train-auc:0.93604	eval-auc:0.67778
[21]	train-auc:0.93696	eval-auc:0.67840
[22]	train-auc:0.93916	eval-auc:0.67838
[23]	train-auc:0.94082	eval-auc:0.67863
[24]	train-auc:0.94757	eval-auc:0.67519
[25]	train-auc:0.95109	eval-auc:0.67647
[26]	trai

In [11]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.7353
Test AUC: 0.6846


# 1. XGBoost

In [10]:
num_seq = 150
num_agents = 11
use_pressing_intensity = False
selected_features_idx = [1, 2, 4, 7]

train_features = []
train_labels = []
for i in range(len(train_dataset)):
    sample = train_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
            
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    train_features.append(feature_vector)
    train_labels.append(y_tensor.item())


test_features = []
test_labels = []
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    x_tensor = sample['features'][..., selected_features_idx][-90:]
    press_intensity = sample['pressing_intensity'][-90:]
    y_tensor = sample['label']
    
    # Flatten the sequence data: shape (sequence_length, num_features) -> (sequence_length*num_features,)
    feature_vector = x_tensor.flatten().numpy()
    
    if use_pressing_intensity:
        if press_intensity.shape[1] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], num_agents-press_intensity.shape[1], press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=1)
        if press_intensity.shape[2] != num_agents:
            pad_tensor = torch.zeros(press_intensity.shape[0], press_intensity.shape[1], num_agents-press_intensity.shape[2])
            press_intensity = torch.cat([press_intensity, pad_tensor], dim=2)
        
        press_vector = press_intensity.flatten().numpy()
        # Concatenate the flattened sequence data with the pressing intensity
        feature_vector = np.concatenate((feature_vector, press_vector))
    test_features.append(feature_vector)
    test_labels.append(y_tensor.item())

X_train = np.array(train_features)
y_train = np.array(train_labels)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_test = np.array(test_features)
y_test = np.array(test_labels)

In [12]:
def print_dataset_distribution(y_train, y_val, y_test):
    def _print_split(name, labels):
        total = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        print(f"{name} Set:")
        print(f"  Total samples: {total}")
        for label, count in zip(unique, counts):
            percent = (count / total) * 100
            print(f"    Label {label}: {count:>5} samples ({percent:5.2f}%)")
        print("-" * 40)

    print("\n📊 Dataset Distribution Summary")
    print("=" * 40)
    _print_split("Train", y_train)
    _print_split("Validation", y_val)
    _print_split("Test", y_test)

print_dataset_distribution(y_train, y_val, y_test)


📊 Dataset Distribution Summary
Train Set:
  Total samples: 11392
    Label 0:  9138 samples (80.21%)
    Label 1:  2254 samples (19.79%)
----------------------------------------
Validation Set:
  Total samples: 2849
    Label 0:  2281 samples (80.06%)
    Label 1:   568 samples (19.94%)
----------------------------------------
Test Set:
  Total samples: 1721
    Label 0:  1358 samples (78.91%)
    Label 1:   363 samples (21.09%)
----------------------------------------


In [9]:
# Create XGBoost DMatrix objects for train and test sets
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

# Set XGBoost training parameters
params = {
    'objective': 'binary:logistic',  # binary classification
    'eval_metric': 'auc',            # evaluation metric: AUC
    'max_depth': 6,                  # maximum depth of trees
    'eta': 0.1,                      # learning rate
    'seed': 42
}

# Specify the watchlist to evaluate performance on training and test sets during training
watchlist = [(dtrain, 'train'), (dtest, 'eval')]
num_rounds = 100

In [10]:
# Train XGBoost model with early stopping on the evaluation set
bst = xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds=10)

/home/work/miniconda/envs/mhl_py311/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-auc:0.72404	eval-auc:0.59755
[1]	train-auc:0.77449	eval-auc:0.61684
[2]	train-auc:0.80143	eval-auc:0.61842
[3]	train-auc:0.82563	eval-auc:0.62081
[4]	train-auc:0.83993	eval-auc:0.62739
[5]	train-auc:0.85611	eval-auc:0.63009
[6]	train-auc:0.86313	eval-auc:0.63397
[7]	train-auc:0.87409	eval-auc:0.63416
[8]	train-auc:0.88935	eval-auc:0.63792
[9]	train-auc:0.89944	eval-auc:0.63648
[10]	train-auc:0.90767	eval-auc:0.63826
[11]	train-auc:0.91626	eval-auc:0.63678
[12]	train-auc:0.92196	eval-auc:0.63293
[13]	train-auc:0.92891	eval-auc:0.63344
[14]	train-auc:0.93481	eval-auc:0.63360
[15]	train-auc:0.93753	eval-auc:0.63349
[16]	train-auc:0.94254	eval-auc:0.63322
[17]	train-auc:0.94739	eval-auc:0.63347
[18]	train-auc:0.94938	eval-auc:0.63573
[19]	train-auc:0.95371	eval-auc:0.63360
[20]	train-auc:0.95596	eval-auc:0.63550


In [ ]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.8027
Test AUC: 0.6491


: 

In [11]:
# W/O Pressing Intensity
# Get predictions on the test set
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)

# Calculate and print evaluation metrics
accuracy = accuracy_score(y_val, y_pred_label)
auc = roc_auc_score(y_val, y_pred)

print("Test Accuracy: {:.4f}".format(accuracy))
print("Test AUC: {:.4f}".format(auc))

Test Accuracy: 0.7989
Test AUC: 0.6465


# 2. SoccerMap / exPress Evaluation

In [39]:
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import json
import os
os.chdir('/home/work/MHL/express-v2')
import argparse # To accept checkpoint path as argument

# Import project modules
# import config  # Import static configurations
from model import PytorchSoccerMapModel # Import Lightning model
from datasets import PressingSequenceDataset, SoccerMapInputDataset 


In [40]:
pl.seed_everything(42, workers=True) # Ensure reproducibility

DATA_PATH = "/data/MHL/pressing-intensity" # Path where pickled datasets are saved
test_dataset = SoccerMapInputDataset(os.path.join(DATA_PATH, "test_dataset.pkl"))

if len(test_dataset) == 0:
    print("Loaded test dataset is empty. Exiting.")

# Custom collate function to handle potential None values from dataset errors
def collate_fn_skip_none(batch):
    batch = list(filter(lambda x: x is not None, batch))
    if not batch: return None
    try: return torch.utils.data.dataloader.default_collate(batch)
    except RuntimeError: return None # Skip batch if collation error

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    # collate_fn=collate_fn_skip_none
)

Seed set to 42


Loading dataset from /data/MHL/pressing-intensity/test_dataset.pkl...


In [41]:
import argparse

parser = argparse.ArgumentParser(description="Train a pressing evaluation model.")
# parser.add_argument("--model_type", type=str, default="soccermap", choices=['soccermap', 'xgboost', 'exPress'], help="Path to the model checkpoint (.ckpt) file saved during training.")
# parser.add_argument("--root_path", type=str, default="/data/MHL/pressing-intensity", help="Path to the data file.")
parser.add_argument("--model_type", type=str, default="soccermap", choices=['soccermap', 'xgboost', 'exPress'], help="Path to the model checkpoint (.ckpt) file saved during training.")
parser.add_argument("--root_path", type=str, default="/data/MHL/pressing-intensity", help="Path to the data file.")
parser.add_argument("--mode", type=str, default="train", choices=['train', 'test'], help="Mode: 'train' or 'test'.")
parser.add_argument("--ckpt_path", type=str, default=None, help="Path to checkpoint file (Required for 'test' mode).")
parser.add_argument("--params_path", type=str, default="params.json", help="Path to the JSON containing configurations.")
parser.add_argument("--seed", type=int, default=42, help="Seed number.")

args = parser.parse_args([])

args.mode = 'test'
args.model_type = "exPress"
args.ckpt_path = "/data/MHL/pressing-intensity/checkpoints/exPress-epoch=28-val_loss=0.49.ckpt"

In [42]:
from components import press


component_dict = {
                    "soccermap": press.SoccerMapComponent,
                    "exPress": press.exPressComponent,
                }

exp = component_dict[args.model_type](args)

Seed set to 42


Configurations loaded from params.json.
